# Bulk Energy Treatment for IrOx Systems
---

R-IrO2: 0.3532      | 0.1766

a-IrO3 (wo H): 0.4279 | 0.2139

a-IrO3 (w H): 0.3909  | 0.1954

# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

sys.path.insert(0,
    os.path.join(
        os.environ["PROJ_irox"],
        "data"))

# #############################################################################
from energetics.dft_energy import Element_Refs

# #############################################################################
# Gas phase molecules
from proj_data_irox import (
    zpe_h2o,
    cv_h2o,
    ts_h2o,

    zpe_h2,
    cv_h2,
    ts_h2)

# #############################################################################
from proj_data_irox import gas_dft_references_dict
gas_dft_refs_i = gas_dft_references_dict[500]

h2o_ref = gas_dft_refs_i["h2o"]
h2_ref = gas_dft_refs_i["h2"]

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_Active_Learning_OER/workflow/energy_treatment_deriv/calc_references


# Script Inputs

In [2]:
T = 298.15

kjmol=96.485
convert_dS = T / (1000 * kjmol)


# DFT Quantities
dft_energy_dict = {
    # FROM AL calculations at 500 eV
    "iro2": -7.047516,
    # "iro2": -7.047426,

    "iro3": -6.469847,
    # "iro3": -6.467450,
    }


# #############################################################################
# IrO2 Experimental thermochemical data #######################################

# # dh_iro2_exp = -242.672 / kjmol  # Barin
# dh_iro2_exp = -2.515126703632689
# # dg_iro2_exp = -188.386 / kjmol  # Barin
# dg_iro2_exp = -1.9524900243561174

from proj_data_irox import (
    dg_iro2_exp,
    dh_iro2_exp)

# #############################################################################
# Entropies ###################################################################

# Reference for IrO2 thermochemical data
# https://onlinelibrary.wiley.com/doi/book/10.1002/9783527619825
# Thermochemical Data of Pure Substances, Third Edition

# rutile-IrO2
S_iro2_solid = 58.576  # J/(mol K)  # Barin
TS_iro2_solid = 0.18098818987407367  # eV


# Ir metal
S_ir_metal = 35.505  # J/(mol K)  # Barin
TS_ir_metal = 0.10969917603772607  # eV


# Oxygen gas entropy reference
# https://webbook.nist.gov/cgi/cbook.cgi?ID=C7782447&Mask=1
# CODATA key values for thermodynamics
# https://www.worldcat.org/title/codata-key-values-for-thermodynamics/oclc/18559968
S_o2_gas = 205.2  # J/(mol K)  # CODATA thermo book
TS_o2_gas = 0.6340921386744053  # eV

# Hydrogen gas entropy

S_h2_gas = 130.68
# TS_h2_gas = S_h2_gas * convert_dS
TS_h2_gas = 0.40381657252422654  # eV

# TdS_irho3=(S_iro2_solid*factor-S_ir_metal-S_o2_gas*3/2-S_h2_gas/2)*convert_dS


# Gas Phase References

In [3]:
# %%capture

e_h2o_r = h2o_ref
e_h2_r = h2_ref

h_h2o_r = e_h2o_r + zpe_h2o + cv_h2o
h_h2_r = e_h2_r + zpe_h2 + cv_h2

g_h2o_r = h_h2o_r + ts_h2o
g_h2_r = h_h2_r + ts_h2

# #############################################################################
Elem_Refs = Element_Refs(
    H2O_dict={
        "electronic_e": e_h2o_r, "enthalpy_e": h_h2o_r, "gibbs_e": g_h2o_r},
    H2_dict={
        "electronic_e": e_h2_r, "enthalpy_e": h_h2_r, "gibbs_e": g_h2_r},
    oxygen_ref="O2",
    hydrogen_ref="H2")

oxy_ref, hyd_ref = Elem_Refs.calc_ref_energies()

Divisor is Energy class instance!!!
Divisor is Energy class instance!!!
Divisor is Energy class instance!!!
Divisor is Energy class instance!!!


# Fitting the Ir metal reference to the experimental formation energy of IrO2

In [4]:
e_iro2_pa = dft_energy_dict["iro2"]
e_iro2 = 3 * e_iro2_pa

PV_i = 0.
h_iro2 = e_iro2 + PV_i

# oxy_ref_h = oxy_ref.enthalpy_e

h_ir_m_fit = h_iro2 - (2 * oxy_ref.enthalpy_e) - dh_iro2_exp  # fit to exp dH

dh_iro2 = h_iro2 - (2 * oxy_ref.enthalpy_e + h_ir_m_fit)

In [5]:
e_iro3_pa = dft_energy_dict["iro3"]
e_iro3 = 4 * e_iro3_pa

PV_i = 0.
h_iro3 = e_iro3 + PV_i

dh_iro3 = h_iro3 - (3 * oxy_ref.enthalpy_e + h_ir_m_fit)

# Calculate Gibbs Free Energy

In [6]:
TdS_iro2 = TS_iro2_solid - TS_ir_metal - TS_o2_gas

# Alternate calculate of TdS for IrO2
# TdS_iro2 = dh_iro2_exp - dg_iro2_exp

dg_iro2 = dh_iro2 - TdS_iro2

# Calculating Gibbs free energy of IrO3 polymorph

In [7]:
# Entropy for WOx from Barrin tables
# WO3: 75.898
# WO2: 50.543

factor_S_wo3_wo2 = 75.898 / 50.543

# adjusted to reflect more O in IrO3 vs IrO2
TS_iro3_solid = factor_S_wo3_wo2 * TS_iro2_solid
# TS_iro3_solid = 0.2139 * 1.0248481872824102 # 1.025 factor scales IrO2 TS to exp. value
print("TS_iro3_solid:", TS_iro3_solid)
TdS_iro3 = TS_iro3_solid - TS_ir_metal - 3 / 2 * TS_o2_gas

print("factor_S_wo3_wo2:", factor_S_wo3_wo2)

factor_S_wo3_wo2 * TS_iro2_solid

TS_iro3_solid: 0.2717812879144974
factor_S_wo3_wo2: 1.5016520586431354


0.2717812879144974

In [8]:
e_iro3_pa = dft_energy_dict["iro3"]
e_iro3 = 4 * e_iro3_pa

PV_i = 0.
h_iro3 = e_iro3 + PV_i

dh_iro3 = h_iro3 - (3 * oxy_ref.enthalpy_e + h_ir_m_fit)
dg_iro3 = dh_iro3 - TdS_iro3

# Calculating Gibbs free energy correction for IrO3H

In [9]:
factor = 4 / 2  # Adjusted to reflect more O H in IriHO3 vs IrO2, phonons needs to be calculated
# TdS_irho3 = factor * TS_iro2_solid - (1) * TS_ir_metal - (3 / 2) * TS_o2_gas - (1 / 2) * TS_h2_gas

TS_H_ads = 0.019  # From Michael Tang H adsorption calculation

TS_irho3_solid = (TS_iro3_solid + TS_H_ads)
print("TS_irho3_solid:", TS_irho3_solid)
TS_irho3_solid = 0.1954 * 1.0248481872824102 # 1.025 factor scales IrO2 TS to exp. value

# TS_irho3_solid = 0.21  # TEMP
# TS_irho3_solid = 0.5  # TEMP
# TS_irho3_solid = 0.19  # TEMP
# TS_irho3_solid = 0.16  # TEMP

TdS_irho3 = TS_irho3_solid - (1) * TS_ir_metal - (3 / 2) * TS_o2_gas - (1 / 2) * TS_h2_gas

TS_irho3_solid: 0.2907812879144974


# Summarizing Data

In [10]:
print(40 * "-")
print("O ref:", oxy_ref.enthalpy_e)
print("H ref:", hyd_ref.enthalpy_e)
print("Ir ref:", h_ir_m_fit)

print(40 * "-")
print("TdS_iro2:", TdS_iro2)
print("TdS_iro3:", TdS_iro3)
print("TdS_irho3:", TdS_irho3)

print(40 * "-")
print("dh_iro2:", dh_iro2)
print("dh_iro3:", dh_iro3)
print("")
print("dg_iro2:", dg_iro2)
print("dg_iro3:", dg_iro3)
print("")
print("dh_iro2 - dh_iro3:", dh_iro2 - dh_iro3)
print("dg_iro2 - dg_iro3:", dg_iro2 - dg_iro3)

print(40 * "-")

----------------------------------------
O ref: -4.64915959
H ref: -3.20624595
Ir ref: -9.32910211636731
----------------------------------------
TdS_iro2: -0.5628031248380577
TdS_iro3: -0.7890560961348366
TdS_irho3: -1.0624903345164645
----------------------------------------
dh_iro2: -2.5151267036326885
dh_iro3: -2.6028071136326894

dg_iro2: -1.9523235787946307
dg_iro3: -1.8137510174978528

dh_iro2 - dh_iro3: 0.08768041000000082
dg_iro2 - dg_iro3: -0.13857256129677786
----------------------------------------


# Collecting variables to save

In [11]:
out_dict = dict(
    h_ref=hyd_ref.enthalpy_e,
    o_ref=oxy_ref.enthalpy_e,
    ir_ref=h_ir_m_fit,
    TdS_iro2=TdS_iro2,
    TdS_iro3=TdS_iro3,
    TdS_irho3=TdS_irho3,
    )

# #######################################################################
import os; import json
directory = "out_data"
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "data.json"), "w") as outfile:
    json.dump(out_dict, outfile, indent=2)
# #######################################################################

In [12]:
# #######################################################################
import json
data_path = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/energy_treatment_deriv/calc_references",
    "out_data/data.json",
    )
with open(data_path, "r") as fle:
    data = json.load(fle)
# #######################################################################

In [13]:
print(20 * "# # ")
print("All done!")
assert False

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
All done!


AssertionError: 